In [3]:
import os
import numpy as np
import nibabel as nib
from dipy.viz import window, actor, colormap

# Archivos
dwi_filepath = 'NoArtifacts_Relaxation.nii.gz' # O .nii
tck_filepath = 'tractograma_multiple.tck'

# Número de streamlines 
num_streamlines_to_show = 1

# Eje y corte para visualizar (0=Sagital, 1=Coronal, 2=Axial)
view_axis = 2 # Axial
slice_index = None # None para el corte central o un número entero

# Validación de Archivos
if not os.path.exists(dwi_filepath):
    print(f"Error: No se encontró el archivo DWI en: {dwi_filepath}")
    exit()
if not os.path.exists(tck_filepath):
    print(f"Error: No se encontró el archivo TCK en: {tck_filepath}")
    exit()

print("Cargando datos...")

# Cargar imagen DWI
img_nii = nib.load(dwi_filepath)
img_data = img_nii.get_fdata()
affine = img_nii.affine


# Accedemos a las streamlines a través del atributo .streamlines
tck_file = nib.streamlines.load(tck_filepath)
# tck_file.streamlines es un generador o lista, lo convertimos a lista si es necesario
# para poder seleccionar las primeras N
all_streamlines = list(tck_file.streamlines) # Convierte a lista para seleccionar

if not all_streamlines:
    print(f"Error: El archivo TCK ({tck_filepath}) no contiene streamlines.")
    exit()

print(f"Archivo TCK cargado. Número total de streamlines: {len(all_streamlines)}")

# Seleccionar un subconjunto de streamlines
if num_streamlines_to_show > len(all_streamlines):
    print(f"Advertencia: Se pidieron {num_streamlines_to_show} streamlines, pero solo hay {len(all_streamlines)}. Mostrando todas.")
    streamlines_subset = all_streamlines
else:
    streamlines_subset = all_streamlines[:num_streamlines_to_show]
    print(f"Mostrando las primeras {len(streamlines_subset)} streamlines.")



######## Visualización

# Usa imagen b0 si es 4D a imagen completa si es 3D
if img_data.ndim == 4:
    # Asume que el primer volumen es b0 o una referencia anatómica
    background_data = img_data[:, :, :, 0]
elif img_data.ndim == 3:
    background_data = img_data
else:
    print(f"Error: Dimensiones de la imagen DWI no soportadas ({img_data.ndim}). Se espera 3D o 4D.")
    exit()

# Determinar el índice del corte central si no se especificó
if slice_index is None:
    slice_index = background_data.shape[view_axis] // 2

print("Creando escena de visualización...")

# Escena con dipy.viz
scene = window.Scene()

# Crear un actor para las streamlines con el mapa de color 'rgb_standard' basado en la orientación xyz
streamlines_actor = actor.line(streamlines_subset, colors = colormap.line_colors(streamlines_subset))

# Crear un actor para mostrar un corte de la imagen de fondo
# value_range ayuda a ajustar el contraste/brillo
data_min, data_max = np.percentile(background_data, [5, 95]) # Rango robusto
slicer_actor = actor.slicer(background_data,
                            affine=affine,
                            value_range=(data_min, data_max), # Ajusta el contraste
                            opacity=0.8) # Ligeramente transparente

# Limites del corte que se muestra en el slicer_actor
slicer_actor.display_extent(
    0, background_data.shape[0] - 1,
    0, background_data.shape[1] - 1,
    0, background_data.shape[2] - 1
)
if view_axis == 0: # Sagital
    slicer_actor.display(slice_index, None, None)
elif view_axis == 1: # Coronal
    slicer_actor.display(None, slice_index, None)
else: # Axial (view_axis == 2)
    slicer_actor.display(None, None, slice_index)

# Añadir los actores a la escena
scene.add(slicer_actor)
scene.add(streamlines_actor)

# Configuracion de la cámara
# scene.set_camera(position=(...), focal_point=(...), view_up=(...))

# Mostrar la escena en una ventana interactiva
print("Mostrando visualización. Cierra la ventana para terminar el script.")
window.show(scene, size=(800, 800), title="Visualización Streamlines sobre DWI")

print("Visualización cerrada.")

Cargando datos...
Archivo TCK cargado. Número total de streamlines: 50
Mostrando las primeras 1 streamlines.
Creando escena de visualización...
Mostrando visualización. Cierra la ventana para terminar el script.
Visualización cerrada.
